In [5]:
from data_processing import *
from feature_extraction import *
from transformations import *

In [10]:
base = "D:/Users/Anthony/Documents/Kaggle-Competitions/KaggleEEG/data_processing/"
train1_path = join(base, 'test')

In [41]:
class Processor(object):

    def __init__(self, list_of_functions, dtrend=None):
        self.list_of_functions = list_of_functions
        self.dtrend = dtrend

    def process_folder(self, train_path, function_name):
        """ Apply function to all files in
        """
        seizure_df = pd.DataFrame()
        failures = []
        results = []
        print(train_path)
        for patient_path in train_path:
            # This is how I speed up processing 4x by making full use of all cores in the CPUs.
            values = [delayed(function_name)('\\'.join([patient_path, f])) for f in listdir(patient_path) if isfile('/'.join([patient_path, f]))]
            result = compute(*values, get=dask.multiprocessing.get)
            results.append(result)
        return results

    def process_file(self, fname):
        """ Apply list of functions to file.

        Each function should return a dataframe with x columns and 1 row. The number of columns is equal to features
        extracted.

        :param fname:
        :param list_of_functions:
        :param dtrend (string):  Must be  'None', 'mean', 'median'
        :return:
        """
        print(fname)
        #base_path, target_path, f = fname.split('\\')
        #path = '\\'.join([base_path, target_path])
        #print('processing', path, f)
        df, sampling_rate, sequence = load_data(join(fname))
        df = self.normalize(df)
        # Determine if this is an inter or preictal dataset and put in corresponding bucket.
        split_string = fname.split('/').pop().replace('.', '_').split('_')
        feature_df_list = []
        for func in self.list_of_functions:
            # Process function and append index columns
            func_result_df = func(df)
            feature_df_list.append(func_result_df)
        feature_df = pd.concat(feature_df_list, 1)
        feature_df = self.append_index(feature_df, split_string)
        return feature_df

    def append_index(self, df, split_string):
        """ Append data set identifier and set index to identifier"""
        df['patient'] = split_string[0]
        df['dataset_id'] = split_string[1]
        df['pre_ictal'] = split_string[2]
        
        return df

    def normalize(self, df):
        """ Normalize data frame.
        """
        return df

In [42]:
test = Processor([extract_means, extract_var])

In [43]:
res = test.process_folder([train1_path], test.process_file)

['D:/Users/Anthony/Documents/Kaggle-Competitions/KaggleEEG/data_processing/test']


In [47]:
type(res[0])

tuple